In [1]:
print("test")

test


In [2]:
Anlage = Standort("muensingen")


In [5]:
from src.ui.day import plot_day
dt = date.today()
fig = plot_day(
    Anlage.load_total_power_of_day(dt),
    Anlage.load_wr_power_of_day(dt),
    *Anlage.calculate_sunrise_times(dt)
)
fig

NameError: name 'date' is not defined

In [4]:
Anlage.calculate_sunrise_times(date.today())

(Timestamp('2026-01-07 08:15:23+0100', tz='Europe/Berlin'),
 Timestamp('2026-01-07 16:45:11+0100', tz='Europe/Berlin'))

In [8]:
#calculate a ttl hash that changes every 5 minutes use timestamp rounded to nearest 5 minutes
# get timestemp float of now and //300 to get the hash
from datetime import datetime
now = int(datetime.now().timestamp() //300)
now


5886911

In [15]:
Anlage.load_daily_yield_this_month()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [19]:
from src.ertrag import update_ertrag
update_ertrag()

Ertragsdaten aktualisiert!


In [ ]:
round(DH.load_daily_yield_this_month()/1_000)

531

In [2]:
dt = date.today()- pd.Timedelta(days=1)
plot_day(DH.load_total_power_of_day(dt),DH.load_wr_power_of_day(dt),*DH.calculate_sunrise_times(dt)).show()

In [4]:
from src.ui.year import plot_calendar_heatmap
Yellows = [[0.0, 'rgb(255, 250, 220)'],[1.0, 'rgb(255, 180, 0)']]

fig_heatmap = plot_calendar_heatmap(DH.load_daily_yield_last_year(), 
                                    date_col='date', 
                                    value_col='value_sum', 
                                    formatting_colorscale=Yellows, 
                                    formatting_locale='de', 
                                    formatting_scale=30,
                                    grid_width=4,
                                    formatting_value_formatter= lambda value: f"⚡ {round(value)} kWh",
                                    highlight_date=date.today()-pd.Timedelta(days=20))
fig_heatmap.show()
    

In [24]:
Reds = [[0.0,'rgb(226, 55, 33)' ],[0.8,'rgb(233, 116, 99)'],[1.0, 'rgb(254, 245, 244)']]
fig_heatmap = plot_calendar_heatmap(DH.calculate_error_statistics(), 
                                    date_col='date', 
                                    value_col='mean_correlation', 
                                    formatting_colorscale=Reds, 
                                    formatting_locale='de', 
                                    formatting_scale=30,
                                    grid_width=4,
                                    formatting_value_formatter= lambda value: f"⚡ {round(value*100)} %",
                                    highlight_date=date.today()-pd.Timedelta(days=20))
fig_heatmap.show()
    

In [7]:
DH.calculate_error_statistics()

,date,corr_3_6,corr_3_1,corr_3_4,corr_3_2,corr_3_5,corr_6_1,corr_6_4,corr_6_2,corr_6_5,...,2_availability,5_availability,total_availability,3_correlation,6_correlation,1_correlation,4_correlation,2_correlation,5_correlation,mean_correlation
0,2024-12-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2024-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2024-12-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2024-12-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2024-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2025-12-13,0.998749,0.997401,0.435695,0.997897,0.997447,0.997960,0.456901,0.998689,0.998032,...,0.989691,0.989691,0.497436,0.998749,0.998749,0.998914,0.456901,0.998914,0.998829,0.908509
361,2025-12-14,0.994828,0.992467,0.849946,0.995190,0.995609,0.995924,0.894181,0.999282,0.995630,...,0.979167,0.958333,0.492308,0.995609,0.999282,0.996455,0.895068,0.999282,0.996455,0.980359
362,2025-12-15,0.965363,0.980403,0.947955,0.946709,0.862605,0.993307,0.981016,0.991853,0.951548,...,1.000000,1.000000,0.502564,0.980403,0.993307,0.993307,0.994341,0.994341,0.974661,0.988393
363,2025-12-16,0.997329,0.999670,0.998999,0.995083,0.995811,0.998031,0.997729,0.998635,0.995940,...,0.989362,0.989362,0.482051,0.999670,0.998635,0.999670,0.999319,0.998635,0.996636,0.998761


In [18]:
from src.leistung import Leistung

leistung = Leistung()
leistung.download_days("muensingen",365)

KeyboardInterrupt: 

In [7]:
# create sample plotly scatter plot
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=[1, 2, 3, 4], y=[10, 11, 12, 13], mode='markers', marker=dict(size=40)))
fig.update_layout(
        plot_bgcolor='white',
        hoverlabel=dict(bgcolor='rgba(255,255,255,0.2)'),
        yaxis=dict(scaleanchor='x', scaleratio=1),
        xaxis=dict(constrain='domain'),
    
        margin=dict(l=0, r=0, t=0, b=0),
        paper_bgcolor='white',
        dragmode=False,
    )
fig.show( config={
                    "displayModeBar": False,
                    "displaylogo": False,
                    "doubleClick": False,
                    "scrollZoom": False,
                    "staticPlot": False,
                    "responsive": False,
                },)

In [12]:
10*55*4

2200